# import Api Keys from Secret Api Keys

In [37]:
from secret_api_keys import Groq_API_Key
from secret_api_keys import riza_api_key
from secret_api_keys import Tavily_api_key

# Set environment varialbes for api keys

In [38]:
import os
os.environ["RIZA_API_KEY"] = riza_api_key
os.environ["TAVILY_API_KEY"] = Tavily_api_key


# import necessary modules for typed dictionaries , state graph and message handiling


In [39]:
from typing import Annotated # import the Annoted class for type hints with additional metadata
from typing_extensions import TypedDict # import the TypeDict class for defining custom typed Dictionaries

from langgraph.graph import StateGraph # import the StateGraph  class for create State Graph
from langgraph.graph.message import add_messages # import the Add_message function for adding messages to list 
from langgraph.prebuilt import ToolNode , tools_condition # import prebuild components like ToolNode And conditions for tool for tools in graph

from langchain_community.tools.tavily_search import TavilySearchResults # import TavilySearchResults class for handiling search results from Travi
from langchain_community.tools.riza.command import  ExecPython # import ExecPython class for Executing python code with riza

In [40]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = Groq_API_Key , model_name=  "llama-3.2-90b-vision-preview")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E8FA609DC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E8FA60C2F0>, model_name='llama-3.2-90b-vision-preview', model_kwargs={}, groq_api_key=SecretStr('**********'))

# Define Tools

In [41]:
tools_tavily =  TavilySearchResults(max_results=2)
tools_code_interpreter = ExecPython()
tools = [tools_tavily,tools_code_interpreter]

In [42]:
llm_with_tools = llm.bind_tools(tools = tools)

In [43]:
response = llm.invoke('''### what is the output of print(['a']+[2])''')
if hasattr(response, 'content'):
    print("AI Response Content:", response.content)
else:
    print("AI Response:", response)

AI Response Content: The output of `print(['a']+[2])` will be:

```python
['a', 2]
```

This is because in Python, you can concatenate lists using the `+` operator. The resulting list will contain all elements from the first list (`['a']`) followed by all elements from the second list (`[2]`).


# Define a typed Dictionary to represent the State Graph

In [44]:
class State(TypedDict):
    messages:Annotated[list,add_messages] # List of messages, With add_message function applied

# Create a StateGraph instance,Specying State type

In [59]:
# Define a chatbot function that takes a state as input and returns a new state with updated messages
def chatBot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}   # Invoke the LLM on the input messages

In [56]:
graph_Builder = StateGraph(State)
graph_Builder.add_node("chatBot",chatBot)
tool_node = ToolNode(tools=tools)
graph_Builder.add_node("tools",tool_node)
graph_Builder.add_conditional_edges(
    "chatBot",
    tools_condition,
)
# Any time a tool is called we return chatBot to decide next step

graph_Builder.add_edge("tools","chatBot")
graph_Builder.set_entry_point("chatBot")
graph = graph_Builder.compile()


# Stream Graph based on user input 

In [61]:
def stream_graph_updates(user_input: str):
    # Create an initial state with the user's input message
    initial_state = {"messages": [('user', user_input)]}  # Fixed key definition

    # Stream updates from the compiled graph
    for event in graph.stream(initial_state):
        # Loop through each event's values (should be only one in this case)
        for value in event.values():
            # Print Assistant's latest message
            print("Assistant:", value["messages"][-1].content)


In [ ]:
while True:
    try:
        user_input = input("user :") # Get input from the user

        if user_input.lower() in ["quit","q","exit"]:
            print("Good Bye!")
            break # Exit the loop

        stream_graph_updates(user_input) # process the input using this graph
    except Exception as e:
        print("Exception is Occured during processing :",e)
        stream_graph_updates(user_input)
        break # exit the loop after the fallback




Assistant: I'm happy to chat with you. Is there something I can help you with or would you like to ask a question?
